### 3 Assignment
### 다음 미완성 코드를 활용해 새로운 data를 학습해보자
* Training data image
  + Shape: (?, 3, 64, 128)
      - 여러장의, RGB, 64x128 size의 이미지라고 가정하자
* Test data image
  + Shape: (?, 3, ?, ?)
      - 여러장의, RGB, size를 알 수 없는 이미지라고 가정하자
* Labels
  + image의 class는 2가지 이다
  + data folder의 구성을 참고하자
* Data folder 위치
  + 노트북 github의 data folder 2개를 적당한 위치에 카피해 사용한다

In [5]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

from google.colab import drive
drive.mount('/content/drive')

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# Model
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 Input shape=(?, 3, 64, 128)
        #    Conv     -> (?, 32, 64, 128)
        #    Pool     -> (?, 32, 32, 64)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # L2 Input shape=(?, 32, 32, 64)
        #    Conv      ->(?, 64, 32, 64)
        #    Pool      ->(?, 64, 16, 32)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # L3 Input shape=(?, 64, 16, 32)
        #    Conv      ->(?, 128, 16, 32)
        #    Pool      ->(?, 128, 8, 16)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # L4 Linear 128x8x16 inputs -> 1024 outputs
        #    Dropout 0.3
        self.layer4 = torch.nn.Sequential(
            torch.nn.Linear(128*8*16, 1024, bias=True),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.3))
        torch.nn.init.xavier_uniform_(self.layer4[0].weight)

        # L5 Linear 1024 inputs -> 2 outputs
        self.layer5 = torch.nn.Linear(1024, 2, bias=True)
        torch.nn.init.xavier_uniform_(self.layer5.weight)


    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC

        out = self.layer4(out)
        out = self.layer5(out)
        return out

# parameters
##### 적당히 수정한다
learning_rate = 0.001
training_epochs = 5

# contruct model
model = CNN().to(device)

# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)   
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train
##### 직접 dataset을 만들어 학습할 때 자주 사용되는 함수이니 기능을 숙지한다
##### pytorch 공식 홈페이지에서 관련 함수의 정보를 직접 찾아보는 것을 추천한다
trans = transforms.Compose([transforms.ToTensor()])
train_data = dsets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/courses/AI_Appl/train_data', transform=trans)
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True)

total_batch = len(data_loader)

##### 학습이 이루어지는 loop
##### 지난 source code를 그대로 사용해도 되지만, 연습삼아 직접 작성해보자
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (3x64x128), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

# test
##### 직접 dataset을 만들어 학습할 때 자주 사용되는 함수이니 기능을 숙지한다
##### pytorch 공식 홈페이지에서 관련 함수의 정보를 직접 찾아보는 것을 추천한다
trans=transforms.Compose([transforms.Resize((64,128)),transforms.ToTensor()])
test_data = dsets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/courses/AI_Appl/test_data', transform=trans)
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

with torch.no_grad():
    test_d = next(iter(test_set))
    X_test = test_d[0].to(device)
    Y_test = test_d[1].to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Learning started. It takes sometime.
[Epoch:    1] cost = 0.20076102
[Epoch:    2] cost = 0.203790709
[Epoch:    3] cost = 0.0138688125
[Epoch:    4] cost = 8.36900799e-05
[Epoch:    5] cost = 0.0
Learning Finished!
Accuracy: 1.0
